In [1]:
%cd ../..

/home/eak/Documents/AI/LLMChat/medivocate


In [2]:
import os
from glob import glob
from src.rag_pipeline.rag_system import RAGSystem

In [3]:
docs_dir="data/docs"
persist_directory_dir="data/chroma_db"
batch_size=64

# Initialize RAG system
rag = RAGSystem(
    docs_dir,
    persist_directory_dir,
    batch_size
)

if len(glob(os.path.join(persist_directory_dir, "*/*.bin"))):
    rag.initialize_vector_store() # vector store initialized
else:
    # Load and index documents
    documents = rag.load_documents()
    rag.initialize_vector_store(documents) # documents

In [4]:
import os
import glob
import re

def parse_questions_answers_with_regex_file(file):
    question_pattern = re.compile(r"<question>(.*?)</question>", re.DOTALL)
    answer_pattern = re.compile(r"<answer>(.*?)</answer>", re.DOTALL)
    qa_list = []

    try:
        with open(file, "r", encoding="utf-8") as f:
            content = f.read()
        
        # Find all questions and answers in the file
        questions = question_pattern.findall(content)
        answers = answer_pattern.findall(content)

        assert len(questions) == len(answers)

        # Pair questions and answers
        qa_list.extend(zip(map(str.strip, questions), map(str.strip, answers)))
    except Exception as e:
        print(f"Error processing file {file}: {e}")
    return qa_list

def parse_questions_answers_with_regex(folder_path):
    """
    Parse question-answer pairs from XML-like text files using regex.

    Args:
        folder_path (str): Path to the folder containing XML-like text files.

    Returns:
        list of tuples: Each tuple contains a question and its corresponding answer.
    """
    # List all text files in the folder
    files = glob.glob(os.path.join(folder_path, "*.txt"))
    qa_list = []

    for file in files:
        qa_list.extend(parse_questions_answers_with_regex_file(file))

    return qa_list


# Parse the question-answer pairs
questions_answers = parse_questions_answers_with_regex("data/evaluation")

len(questions_answers)

45

In [5]:
from src.utilities.llm_models import get_llm_model_embedding

embedder = get_llm_model_embedding()

import numpy as np

def cosine_similarity(expected, proposed):
    # Encode the input sentences to embeddings
    embeddings = np.array(embedder.embed_documents([
        expected,
        proposed
    ]))
    
    # Compute the dot product and norms
    dot_product = np.dot(embeddings[0], embeddings[1])
    norm_expected = np.linalg.norm(embeddings[0])
    norm_proposed = np.linalg.norm(embeddings[1])
    
    # Compute cosine similarity
    similarity = dot_product / (norm_expected * norm_proposed)
    return similarity


In [6]:
from tqdm import tqdm

scores = []
for query, correct in tqdm(questions_answers):
    try:
        response = rag.query(query)
        score = cosine_similarity(
            correct,
            response["answer"],
        )
        scores.append({
            "score": float(score),
            "correct": correct,
            "answer": response["answer"],
            "query": query
        })
    except:
        pass

100%|██████████| 45/45 [03:15<00:00,  4.35s/it]


In [7]:
scores[:3]

[{'score': 0.7327400757557035,
  'correct': 'The specialization of certain groups in specific crafts allowed for a more diverse range of products, from textiles and jewelry to agricultural surplus and artisanal goods, which supported both local consumption and trade.',
  'answer': 'Specialization in crafts such as tanning, weaving, and pottery contributed significantly to economic diversity within the Hawsa region by offering a range of products that could be traded. These crafts provided raw materials for other industries and created demand for their finished goods. For instance, leather from tanned hides was used in various clothing items, while woven fabrics were highly valued due to their quality. Pottery, with its diverse uses including storage vessels, cooking pots, and decorative items, also played a crucial role in the economy by meeting local needs and potentially being traded for other goods. This specialization allowed the Hawsa region to diversify its economic activities be

In [8]:
import json

json.dump(scores, open("data/evaluation_result.json", "w"))

In [9]:
np.mean([
    score["score"] for score in scores
])

0.6354498400520328

In [10]:
np.max([
    score["score"] for score in scores
])

0.8211647458149876

In [11]:
np.min([
    score["score"] for score in scores
])

0.2698309741436685